In [1]:
#Cleaning up  https://github.com/dasotelo/Python_Projects/blob/master/Wine_Bag_Boost.py
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

In [2]:
wine_df=pd.read_csv('winemag-data_first150k.csv')
wine_df = wine_df[['country','points','price','region_1', 'variety','winery']]
wine_df=wine_df.dropna(axis=0,how='any')

In [3]:
qualityCutoff = 90
pointRange = wine_df.points.unique().tolist()
qualityMap = {}
for point in pointRange:
    if point >= qualityCutoff:
        qualityMap[point] = str(qualityCutoff) + "+"
    else:
        qualityMap[point] = "<" + str(qualityCutoff)
wine_df['Quality']=wine_df.points.map(qualityMap)
wine_df=wine_df.drop('points',axis=1)

In [4]:
wine_df.head()

,country,price,region_1,variety,winery,Quality
0,US,235.0,Napa Valley,Cabernet Sauvignon,Heitz,90+
1,Spain,110.0,Toro,Tinta de Toro,Bodega Carmen Rodríguez,90+
2,US,90.0,Knights Valley,Sauvignon Blanc,Macauley,90+
3,US,65.0,Willamette Valley,Pinot Noir,Ponzi,90+
4,France,66.0,Bandol,Provence red blend,Domaine de la Bégude,90+


In [5]:
# Prepare data by labeling/encoding
def labelCategoryData(df,label):
    labels = LabelEncoder().fit(np.unique(df[label].values))
    df[label+'_enc'] = labels.transform(df[label])
    return(df)

In [6]:
labels = ['country','region_1','variety','winery']
for label in labels:
    wine_df = labelCategoryData(wine_df,label)
wine_df=wine_df.drop(['country','region_1','variety','winery'],axis=1)

In [7]:
wine_df.head()

,price,Quality,country_enc,region_1_enc,variety_enc,winery_enc
0,235.0,90+,6,708,40,5910
1,110.0,90+,5,1034,402,1008
2,90.0,90+,6,505,342,7294
3,65.0,90+,6,1183,298,8901
4,66.0,90+,3,62,309,4422


In [8]:
wine_train,wine_test=train_test_split(wine_df,test_size=0.90)
x=wine_train.loc[:,['price','country_enc','region_1_enc','variety_enc','winery_enc']]
y=wine_train['Quality']
clf=DecisionTreeClassifier(max_features=None)
clf=clf.fit(x,y)

In [9]:
# Report classification results.  training dataset first, then test.  
train_error=y==clf.predict(x)
test_error=wine_test['Quality']==clf.predict(wine_test.loc[:,['price','country_enc','region_1_enc','variety_enc','winery_enc']])
print('@@@@@@@@@@@@@@@@@@@@@@@@@')
print('CART w/ #leaf nodes = ',clf.tree_.node_count) 
print('   ',clf.n_features_,' features out of: 4 features')                          
print('   training accuracy: ','{:.1%}'.format(sum(train_error)/len(
    train_error)))
print('   test accuracy: ','{:.1%}'.format(sum(test_error)/len(test_error)))

@@@@@@@@@@@@@@@@@@@@@@@@@
CART w/ #leaf nodes =  5313
    5  features out of: 4 features
   training accuracy:  98.9%
   test accuracy:  73.3%


In [ ]:
# Control the number of n_estimators in ensemble functions
max_n_ests=25

# Create dataframe to record results of ensembles.
results=pd.DataFrame([],columns=list(['type','n_leaf','n_est', 'train_acc','test_acc']))

# Train bagging ensemble on iterations of n_estimators=i
# and iterations of stump max_leaf_nodes=j
for j in [500,2000,8000,99999]:
    clf_stump=DecisionTreeClassifier(max_features=None,max_leaf_nodes=j)
    for i in np.arange(1,max_n_ests):
        #print(i)
        baglfy=BaggingClassifier(base_estimator=clf_stump,n_estimators=i,
            max_samples=1.0)
        baglfy=baglfy.fit(x,y)
        bag_tr_err=y==baglfy.predict(x)
        bag_tst_err=wine_test['Quality']==baglfy.predict( \
            wine_test.loc[:,['price','country_enc','region_1_enc','variety_enc','winery_enc']])
        run_rslt=pd.DataFrame([['bag',j,i,sum(bag_tr_err)/len(bag_tr_err),
            sum(bag_tst_err)/len(bag_tst_err)]],
            columns=list(['type','n_leaf','n_est','train_acc','test_acc']))
        results=results.append(run_rslt)

In [ ]:
# Train boosting ensemble on iterations of n_estimators=i
# and iterations of stump max_leaf_nodes=j
for j in [500,2000,8000,99999]:
    clf_stump=DecisionTreeClassifier(max_features=None,max_leaf_nodes=j)
    for i in np.arange(1,max_n_ests):
        #print(i)
        bstlfy=AdaBoostClassifier(base_estimator=clf_stump,n_estimators=i)
        bstlfy=bstlfy.fit(x,y)
        bst_tr_err=y==bstlfy.predict(x)
        bst_tst_err=wine_test['Quality']==bstlfy.predict( \
            wine_test.loc[:,['price','country_enc','region_1_enc','variety_enc','winery_enc']])
        run_rslt=pd.DataFrame([['bst',j,i,sum(bst_tr_err)/len(bst_tr_err),
            sum(bst_tst_err)/len(bst_tst_err)]],
            columns=list(['type','n_leaf','n_est','train_acc','test_acc']))
        results=results.append(run_rslt)

In [ ]:
# Plot Bagging accuracy results on test data
# 500 leaf stumps
plt.plot(results.loc[((results.type=='bag')&(results.n_leaf==500)), \
    ['n_est']],results.loc[((results.type=='bag')&(results.n_leaf==500)), \
    ['test_acc']],linestyle='--',linewidth=2,color='#4da6ff', \
    label='Bagging w/ 500 leaf stump')
# 2000 leaf stumps
plt.plot(results.loc[((results.type=='bag')&(results.n_leaf==2000)), \
    ['n_est']],results.loc[((results.type=='bag')&(results.n_leaf==2000)), \
    ['test_acc']],linestyle='--',linewidth=2,color='#0080ff', \
    label='Bagging w/ 2000 leaf stump')
# 8000 leaf stumps
plt.plot(results.loc[((results.type=='bag')&(results.n_leaf==8000)), \
    ['n_est']],results.loc[((results.type=='bag')&(results.n_leaf==8000)), \
    ['test_acc']],linestyle='--',linewidth=2,color='#0059b3', \
    label='Bagging w/ 8000 leaf stump')
# Full Classification Trees (no early termination)
plt.plot(results.loc[((results.type=='bag')&(results.n_leaf==99999)), \
    ['n_est']],results.loc[((results.type=='bag')&(results.n_leaf==99999)), \
    ['test_acc']],linestyle='--',linewidth=2,color='#003366', \
    label='Bagging w/ full tree')
# Plot test accuracy of baseline classification tree
clf_test_acc=sum(test_error)/len(test_error)
plt.plot([1,max_n_ests],[clf_test_acc,clf_test_acc],color='k', \
    label='Baseline classification tree')
plt.legend(fontsize=8)
plt.title('Bagging Test Sample Accuracy on n_estimators')
plt.ylim([results.loc[results.type=='bag',['test_acc']].values.min()-0.01, \
    results.loc[results.type=='bag',['test_acc']].values.max()+0.01])
plt.ylabel('Test Accuracy%')
plt.xlabel('n_estimators')
plt.show()


In [ ]:
# Plot Boosting accuracy results on test data
# 500 leaf stumps
plt.plot(results.loc[((results.type=='bst')&(results.n_leaf==500)), \
    ['n_est']],results.loc[((results.type=='bst')&(results.n_leaf==500)), \
    ['test_acc']],linestyle='--',linewidth=2,color='#ff704d', \
    label='Boosting w/ 500 leaf stump')
# 2000 leaf stumps
plt.plot(results.loc[((results.type=='bst')&(results.n_leaf==2000)), \
    ['n_est']],results.loc[((results.type=='bst')&(results.n_leaf==2000)), \
    ['test_acc']],linestyle='--',linewidth=2,color='#ff3300', \
    label='Boosting w/ 2000 leaf stump')
# 8000 leaf stumps
plt.plot(results.loc[((results.type=='bst')&(results.n_leaf==8000)), \
    ['n_est']],results.loc[((results.type=='bst')&(results.n_leaf==8000)), \
    ['test_acc']],linestyle='--',linewidth=2,color='#b32400', \
    label='Boosting w/ 8000 leaf stump')
# Full Classification Trees (no early termination)
plt.plot(results.loc[((results.type=='bst')&(results.n_leaf==99999)), \
    ['n_est']],results.loc[((results.type=='bst')&(results.n_leaf==99999)), \
    ['test_acc']],linestyle='--',linewidth=2,color='#661400', \
    label='Boosting w/ full tree')
# Plot test accuracy of baseline classification tree
plt.plot([1,max_n_ests],[clf_test_acc,clf_test_acc],color='k', \
    label='Baseline classification tree')
plt.legend(fontsize=8)
plt.title('Boosting Test Sample Accuracy on n_estimators')
plt.ylim([results.loc[results.type=='bst',['test_acc']].values.min()-0.01, \
    results.loc[results.type=='bst',['test_acc']].values.max()+0.01])
plt.ylabel('Test Accuracy%')
plt.xlabel('n_estimators')
plt.show()

In [ ]:
# ROC curve for baseline classification tree
clf_probs=clf.predict_proba(wine_test.loc[:,['price','country_enc','region_1_enc','variety_enc','winery_enc']])
fpr1,tpr1,thr1=roc_curve(np.where(wine_test['Quality']=='90+',1.,0.), \
    clf_probs[:,0])
# ROC curve for bagging ensemble using full classification trees
bag_probs=baglfy.predict_proba(wine_test.loc[:,['price','country_enc','region_1_enc','variety_enc','winery_enc']])
fpr2,tpr2,thr2=roc_curve(np.where(wine_test['Quality']=='90+',1.,0.), \
    bag_probs[:,0])
# ROC curve for boosting ensemble using full classification trees
bst_probs=bstlfy.predict_proba(wine_test.loc[:,['price','country_enc','region_1_enc','variety_enc','winery_enc']])
fpr3,tpr3,thr3=roc_curve(np.where(wine_test['Quality']=='90+',1.,0.), \
    bst_probs[:,0])

In [ ]:
# Plot ROC Curves
plt.plot(fpr1,tpr1,color='#4d4d33',label='Baseline CART')
plt.plot(fpr2,tpr2,color='#0080ff',label='Bagging Ensemble')
plt.plot(fpr3,tpr3,color='#ff3300',label='Boosting Ensemble')
plt.plot([0.,1.],[0.,1.],color='k',linestyle='--')
plt.title('ROC Curves for 90+ Point Wine Classification')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.legend(fontsize=8)
plt.show()